# Instructions

To produce the interactive graphics in this notebook, go up to the top menu and select Run -> Run All Cells.  This will generate two interactive graphics below.

The first graphic will be a layered image with the bottom layer being an infrared image, the middle layer being an OCT image, and the top layer being the difference in visual field measurements for the OCT-plotted date vs baseline date.

The second graphic is a sequence of images.  The first (left-most) image is a baseline image similar to the first graphic, and the second image shows the evolution in time of that same graphic.  The third and fourth images plot the decrease and increase in OCT measured values relative to baseline, respectively, as the middle layer of the image.

Note:
* Both graphics have menus to select different data options, as well as to vary the opacity of the middle and top layers.
* There are interactive buttons to play an animation.  To play a time sequence of images, simply click on the play button.

## Importing libraries
The following cell will not produce any output

In [ ]:
import eyeviz
import eyeviz2
import h5py
import numpy as np
import os
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interactive, VBox, HBox, interactive_output
import pandas as pd
from IPython.display import display, clear_output
from matplotlib.colors import LinearSegmentedColormap
cmap1 = LinearSegmentedColormap.from_list('rg',["r", "w", "g"], N=256)

## Graphic #1: the layered image

In [ ]:
h5f = h5py.File(eyeviz.h5files[0],'r')
def ploteyew(patient='070',eyeside='L',t=0,gridpts=640,tness='GCIPL',opacity1=1.0,opacity2=1.0):
    eyeviz.ploteye(h5f,patient,eyeside,t,gridpts,tness,opacity1,opacity2)
    
d1 = widgets.Dropdown(options=eyeviz.numsunique,description='Patient',style={'description_width': 'initial'})
d2 = widgets.Dropdown(options=['L','R'],description='Eyeside',style={'description_width': 'initial'})
d3 = widgets.Dropdown(options=eyeviz.thickunique,description='Layer',value='GCIPL',style={'description_width': 'initial'})
d4 = widgets.Dropdown(options=eyeviz.yunique,description='Grid Pts',style={'description_width': 'initial'})
s1 = widgets.IntSlider(min=0,max=len(h5f['070/L'].keys())-1,description='Current Image #:',style={'description_width': 'initial'})
s2 = widgets.FloatSlider(value=1.0,min=0.0,max=1.0,step=0.1,description='Opacity OCT:',style={'description_width': 'initial'})
s3 = widgets.FloatSlider(value=1.0,min=0.0,max=1.0,step=0.1,description='Opacity VF_diff:',style={'description_width': 'initial'})

def valued1_changed(change):
    global h5f
    h5f.close()
    h5f = h5py.File('data/'+change.new+'-'+d3.value+'.hdf5','r')
    if len(h5f.keys()) == 0:
        s1.max = 0
    elif d2.value not in h5f[change.new].keys():
        s1.max = 0
    else:
        s1.max = max(0,len(h5f[change.new+'/'+d2.value].keys())-1)
    s1.value = 0
d1.observe(valued1_changed, 'value')

def valued2_changed(change):
    if len(h5f.keys()) == 0:
        s1.max = 0
    elif change.new not in h5f[d1.value].keys():
        s1.max = 0
    else:
        s1.max = max(0,len(h5f[d1.value+'/'+change.new].keys())-1)
    s1.value = 0
d2.observe(valued2_changed, 'value')

def valued3_changed(change):
    global h5f
    h5f.close()
    h5f = h5py.File('data/'+d1.value+'-'+change.new+'.hdf5','r')
    if len(h5f.keys()) == 0:
        s1.max = 0
    elif d2.value not in h5f[d1.value].keys():
        s1.max = 0
    else:
        s1.max = max(0,len(h5f[d1.value+'/'+d2.value].keys())-1)
    s1.value = 0
d3.observe(valued3_changed, 'value')

play = widgets.Play(value=0,min=0,max=s1.max,step=1,interval=1500,description="Press play",disabled=False)
left_box = VBox([d1,d2,d3,d4])
middle_box = VBox([s1,s2,s3])
right_box = VBox([play])
controls = HBox([left_box, middle_box, right_box])

plotwidget = interactive_output(ploteyew, {'patient':d1,
                                            'eyeside':d2,
                                            'tness':d3,
                                            'gridpts':d4,
                                            't':s1,
                                            'opacity1':s2,
                                            'opacity2':s3});

widgets.jslink((play, 'value'), (s1, 'value'))
totalwidget = widgets.VBox([controls,plotwidget])
display(totalwidget)

## Graphic #2: the layered image plus plotted difference in OCT values

In [ ]:
h5f = h5py.File(eyeviz.h5files[0],'r')
def ploteyew2(patient='070',eyeside='L',t=0,gridpts=640,tness='GCIPL',opacity1=1.0,opacity2=1.0):
    fig = eyeviz2.ploteye2(h5f,patient,eyeside,t,gridpts,tness,opacity1,opacity2)        

d1 = widgets.Dropdown(options=eyeviz.numsunique,description='Patient',style={'description_width': 'initial'})
d2 = widgets.Dropdown(options=['L','R'],description='Eyeside',style={'description_width': 'initial'})
d3 = widgets.Dropdown(options=eyeviz.thickunique,description='Layer',value='GCIPL',style={'description_width': 'initial'})
d4 = widgets.Dropdown(options=eyeviz.yunique,description='Grid Pts',style={'description_width': 'initial'})
s1 = widgets.IntSlider(min=0,max=len(h5f['070/L'].keys())-1,description='Current Image #:',style={'description_width': 'initial'})
s2 = widgets.FloatSlider(value=1.0,min=0.0,max=1.0,step=0.1,description='Opacity OCT:',style={'description_width': 'initial'})
s3 = widgets.FloatSlider(value=1.0,min=0.0,max=1.0,step=0.1,description='Opacity VF_diff:',style={'description_width': 'initial'})

def valued1_changed(change):
    global h5f
    h5f.close()
    h5f = h5py.File('data/'+change.new+'-'+d3.value+'.hdf5','r')
    if len(h5f.keys()) == 0:
        s1.max = 0
    elif d2.value not in h5f[change.new].keys():
        s1.max = 0
    else:
        s1.max = max(0,len(h5f[change.new+'/'+d2.value].keys())-1)
    s1.value = 0
d1.observe(valued1_changed, 'value')

def valued2_changed(change):
    if len(h5f.keys()) == 0:
        s1.max = 0
    elif change.new not in h5f[d1.value].keys():
        s1.max = 0
    else:
        s1.max = max(0,len(h5f[d1.value+'/'+change.new].keys())-1)
    s1.value = 0
d2.observe(valued2_changed, 'value')

def valued3_changed(change):
    global h5f
    h5f.close()
    h5f = h5py.File('data/'+d1.value+'-'+change.new+'.hdf5','r')
    if len(h5f.keys()) == 0:
        s1.max = 0
    elif d2.value not in h5f[d1.value].keys():
        s1.max = 0
    else:
        s1.max = max(0,len(h5f[d1.value+'/'+d2.value].keys())-1)
    s1.value = 0
d3.observe(valued3_changed, 'value')

play = widgets.Play(value=0,min=0,max=s1.max,step=1,interval=1500,description="Press play",disabled=False)
left_box = VBox([d1,d2,d3,d4])
middle_box = VBox([s1,s2,s3])
right_box = VBox([play])
controls = HBox([left_box, middle_box, right_box])

plotwidget = interactive_output(ploteyew2, {'patient':d1,'eyeside':d2,'tness':d3,'gridpts':d4,
         't':s1,'opacity1':s2,'opacity2':s3});

widgets.jslink((play, 'value'), (s1, 'value'))
totalwidget = widgets.VBox([controls,plotwidget])
display(totalwidget)